# PHE SEIR Model

In this notebook we present compare the model built by Public Health England in collaboration with University of Cambridge, using baseline contact matrices with a simple SEIR. We aim to see if the PHE model can be reduced to a simple SEIR model for appropriate choices of parameters to check how well the PHE model has been reproduced.

The analyses are run for:
 - Dates: **15 Feb 2020** - **15 May 2020**, using toy data;
 - PHE regions of interest: **South West**.

We consider only **1 region** and **1 age group**. To be able to collapse the PHE model in a simpler SEIR version, we will have to choose the parameters such that:

$$
\begin{align*}
    \frac{dS(t)}{dt} &= -\lambda_t S(t) \\
    \frac{dE^1(t)}{dt} &= \lambda_t S(t) - \frac{2}{d_L} E^1(t) \\
    \frac{dE^2(t)}{dt} &= \frac{2}{d_L} E^1(t) - \frac{2}{d_L} E^2(t) \\
    \frac{dI^1(t)}{dt} &= \frac{2}{d_L} E^2(t) - \frac{2}{d_I} I^1(t) \\
    \frac{dI^2(t)}{dt} &= \frac{2}{d_I} I^1(t) - \frac{2}{d_I} I^2(t) \\
    \frac{dR(t)}{dt} &= \frac{2}{d_I} I^2(t)
\end{align*}
$$

becomes

$$
\begin{align*}
    \frac{dS(t)}{dt} &= - \beta I(t) S(t) \\
    \frac{dE(t)}{dt} &= \beta I(t) S(t) - \kappa E(t) \\
    \frac{dI(t)}{dt} &= \kappa E(t) - \gamma I(t) \\
    \frac{dR(t)}{dt} &= \gamma I(t)
\end{align*}
$$

We have two compartments for each of the exposed and infectious in the PHE model, compared to only one in the SEIR one. Hence, we will assume a collapse of this structure, by using the following notation: $E(t) = E^1(t) + E^2(t)$ and $I(t) = I^1(t) + I^2(t)$.

The parameter $\lambda_{t} = 1 - (1-b_t)^{I^1(t) + I^2(t)}$ will need to become equivalent in shape to $\beta I(t)$. We will therefore assume the conditions of the _binomial approximation_:

$$\lambda_{t} = 1 - (1-b_t)^{I^1(t) + I^2(t)} \approx 1 - (1-b_t(I^1(t) + I^2(t))) = 1 - (1-b_t I(t)) = b_t I(t)$$

i.e. $|b_t| < 1$ and $|b_t I(t)| \ll 1$ .

The $b_t$ is computed according to the following formula:

$$b_t = \frac{\beta_t R_0}{R_0^{*}} \tilde{C}_t$$

where $\beta_t$ account for mis-specification of the changing scale of transmission over time, $R_0$ is the initial reproduction number, $\tilde{C}_t$ is the current contact matrix, $R_0^{*}$ is the dominant eigenvalue of $S_0 \tilde{C}_0 d_I$, $S_0$ is the initial number of susceptibles and $d_I$ is the average time an individual is infectious. Since the matrices are one-dimensional, $R_0^{*} = S_0 \tilde{C}_0 d_I$. We have no temporal variations in our parameters of the SEIR model, hence we take the contact matrices for the PHE model to be constant throughout and $\beta_t = 1, \forall t$ so that the transmission parameter becomes

$$b_t = \frac{R_0}{S_0 d_I}$$

This is satisfying the _binomial approximation_ conditions $|\frac{R_0 I(t)}{S_0 d_I}| \ll 1$ as the number of infectives is much smaller than that of initial susceptibles.

Also, since the rate of transmission between the compartments are defined as the inverse of the average time spent in the compartment, the system of equations of the PHE model illustrate that an individual spends on average in each of the exposed and infectious compartments are:

- $E^1$ and $E^2$: $\frac{d_L}{2}$ each, so a total average of $d_L$ as exposed;
- $I^1$ and $I^2$: $\frac{d_I}{2}$ each, so a total average of $d_I$ as infectious.

Hence, for the SEIR model, the parameters are defined as:

- $\beta = \frac{R_0}{S_0 d_I}$ ;
- $\kappa = \frac{1}{d_L}$ ;
- $\gamma = \frac{1}{d_I}$ .

*The PHE model is built by Public Health England in collaboration with University of Cambridge.*

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import epimodels as em
import pints
import matplotlib
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the PHE Model

In [2]:
# Populate the model
total_days =  90
regions = ['SW']
age_groups = ['0-75+']

matrices_region = []

### Fixed
# Initial state of the system
weeks_matrices_region = []
for r in regions:
    path = os.path.join('../../data/final_contact_matrices/BASE.csv')
    region_data_matrix = np.average(pd.read_csv(path, header=None, dtype=np.float64)) * np.ones((len(age_groups), len(age_groups)))
    regional = em.RegionMatrix(r, age_groups, region_data_matrix)
    weeks_matrices_region.append(regional)

matrices_region.append(weeks_matrices_region)

contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
matrices_contact = [contacts]

# Matrices contact
time_changes_contact = [1]
time_changes_region = [1]

### Set the parameters and initial conditions of the model and bundle everything together

In [3]:
# Instantiate model
phe_model = em.PheSEIRModel()

# Set the region names, contact and regional data of the model
phe_model.set_regions(regions)
phe_model.set_age_groups(age_groups)
phe_model.read_contact_data(matrices_contact, time_changes_contact)
phe_model.read_regional_data(matrices_region, time_changes_region)

# Set regional and time dependent parameters
regional_parameters = em.PheRegParameters(
    model=phe_model,
    initial_r=[2.2164],
    region_index=1,
    betas=[[1] * np.arange(1, total_days, 1).shape[0]],
    times=np.arange(1, total_days, 1).tolist()
)

# Set ICs parameters
ICs = em.PheICs(
    model=phe_model,
    susceptibles_IC=[[5000]],
    exposed1_IC=[[0]],
    exposed2_IC=[[0]],
    infectives1_IC=[[50]],
    infectives2_IC=[[20]],
    recovered_IC=[[0]]
)

# Set disease-specific parameters
disease_parameters = em.PheDiseaseParameters(
    model=phe_model,
    dL=4,
    dI=4
)

# Set other simulation parameters
simulation_parameters = em.PheSimParameters(
    model=phe_model,
    delta_t=0.5,
    method='RK45'
)

# Set all parameters in the controller
phe_parameters = em.PheParametersController(
    model=phe_model,
    regional_parameters=regional_parameters,
    ICs=ICs,
    disease_parameters=disease_parameters,
    simulation_parameters=simulation_parameters
)

### Define SEIR model

In [4]:
class SEIRModel(pints.ForwardModel):
    r"""
    ODE model: deterministic SEIR
    The SEIR Model has four compartments:
    susceptible individuals (:math:`S`),
    exposed but not yet infectious (:math:`E`),
    infectious (:math:`I`) and recovered (:math:`R`):

    .. math::
        \frac{dS(t)}{dt} = -\beta S(t)I(t),
    .. math::
        \frac{dE(t)}{dt} = \beta S(t)I(t) - \kappa E(t),
    .. math::
        \frac{dI(t)}{dt} = \kappa E(t) - \gamma I(t),
    .. math::
        \frac{dR(t)}{dt} = \gamma I(t),

    where :math:`S(0) = S_0, E(0) = E_0, I(O) = I_0, R(0) = R_0`
    are also parameters of the model.

    Extends :class:`ForwardModel`.
    """

    def __init__(self):
        super(SEIRModel, self).__init__()

        # Assign default values
        self._output_names = ['S', 'E', 'I', 'R', 'Incidence']
        self._parameter_names = [
            'S0', 'E0', 'I0', 'R0', 'alpha', 'beta', 'gamma'
        ]
        # The default number of outputs is 5,
        # i.e. S, E, I, R and Incidence
        self._n_outputs = len(self._output_names)
        # The default number of outputs is 7,
        # i.e. 4 initial conditions and 3 parameters
        self._n_parameters = len(self._parameter_names)

        self._output_indices = np.arange(self._n_outputs)

    def n_outputs(self):
        # Return the number of outputs
        return self._n_outputs

    def n_parameters(self):
        # Return the number of parameters
        return self._n_parameters

    def output_names(self):
        # Return the (selected) output names
        names = [self._output_names[x] for x in self._output_indices]
        return names

    def parameter_names(self):
        # Return the parameter names
        return self._parameter_names

    def set_outputs(self, outputs):
        # Check existence of outputs
        for output in outputs:
            if output not in self._output_names:
                raise ValueError(
                    'The output names specified must be in correct forms')

        output_indices = []
        for output_id, output in enumerate(self._output_names):
            if output in outputs:
                output_indices.append(output_id)

        # Remember outputs
        self._output_indices = output_indices
        self._n_outputs = len(outputs)

    def _right_hand_side(self, t, y, c):
        # Assuming y = [S, E, I, R] (the dependent variables in the model)
        # Assuming the parameters are ordered like
        # parameters = [S0, E0, I0, R0, beta, kappa, gamma]
        # Let c = [beta, kappa, gamma]
        #  = [parameters[0], parameters[1], parameters[2]],
        # then beta = c[0], kappa = c[1], gamma = c[2]

        # Construct the derivative functions of the system of ODEs

        s, e, i, _ = y
        beta, kappa, gamma = c
        dydt = [-beta * s * i, beta * s * i - kappa * e,
                kappa * e - gamma * i, gamma * i]

        return dydt

    def simulate(self, parameters, times):

        # Define time spans, initial conditions, and constants
        y_init = list(deepflatten(parameters[:4]))
        c = list(deepflatten(parameters[4:]))

        # Solve the system of ODEs
        sol = solve_ivp(lambda t, y: self._right_hand_side(t, y, c),
                        [times[0], times[-1]], y_init, t_eval=times)

        output = sol['y']

        # Total infected is infectious 'i' plus recovered 'r'
        total_infected = output[2, :] + output[3, :]

        # Number of incidences is the increase in total_infected
        # between the time points (add a 0 at the front to
        # make the length consistent with the solution
        n_incidence = np.zeros(len(times))
        n_incidence[1:] = total_infected[1:] - total_infected[:-1]

        # Append n_incidence to output
        # Output is a matrix with rows being S, E, I, R and Incidence
        output = np.vstack(tup=(output, n_incidence))

        # Get the selected outputs
        output = output[self._output_indices, :]

        return output.transpose()


### Set the parameters and initial conditions of the model and bundle everything together

In [5]:
# Instantiate model
seir_model = SEIRModel()

# Set all parameters
S0 = np.int64(np.asarray(phe_parameters.ICs.susceptibles))
E0 = np.int64(np.asarray(phe_parameters.ICs.exposed1)) + np.int64(np.asarray(phe_parameters.ICs.exposed2))
I0 = np.int64(np.asarray(phe_parameters.ICs.infectives1)) + np.int64(np.asarray(phe_parameters.ICs.infectives2))
R0 = np.int64(np.asarray(phe_parameters.ICs.recovered))
beta = np.asarray(phe_parameters.regional_parameters.initial_r) / (S0 * phe_parameters.disease_parameters.dI)
kappa = 1/phe_parameters.disease_parameters.dL
gamma = 1/phe_parameters.disease_parameters.dI

seir_parameters = [S0, E0, I0, R0, beta, kappa, gamma]

In [6]:
seir_parameters

[array([[5000]]),
 array([[0]]),
 array([[70]]),
 array([[0]]),
 array([[0.00011082]]),
 0.25,
 0.25]

### Simulate for one of the regions: **South West**

In [7]:
# Simulate the PHE model using the ODE solver from scipy
ext_output_phe_model = phe_model.simulate(phe_parameters)
output_phe_model = np.stack((ext_output_phe_model[:, 0], ext_output_phe_model[:, 1] + ext_output_phe_model[:, 2], ext_output_phe_model[:, 3] + ext_output_phe_model[:, 4], ext_output_phe_model[:, 5], ext_output_phe_model[:, 6]), axis=1)

# Simulate the SEIR model using the ODE solver from scipy
output_seir_model = seir_model.simulate(seir_parameters, times=phe_parameters.regional_parameters.times)

## Plot the comparments of the two models against each other
### Setup ``plotly`` and default settings for plotting

In [8]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']

# Group outputs together
outputs = [output_phe_model, output_seir_model]

### Plot the comparments of the two models against each other

In [9]:
# Trace names - represent the solver used for the simulation
trace_name = ['PHE model', 'SEIR model']

# Compartment list - type and age
comparments = []
for n in seir_model.output_names():
    comparments.append('{}'.format(n))

fig = go.Figure()
fig = make_subplots(
    rows=int(np.ceil(len(comparments)/2)), cols=2, subplot_titles=tuple('{}'.format(comparment) for comparment in comparments)
)

# Plot for each comparment
for c, comparment in enumerate(comparments):
    # Plot (line plot for each solver method for each age)
    for a, age in enumerate(age_groups):
        if c != 0:
            for o, out in enumerate(outputs):
                fig.add_trace(
                    go.Scatter(
                        y=out[:, c*len(age_groups)+a],
                        x=phe_parameters.regional_parameters.times,
                        mode='lines',
                        name=trace_name[o],
                        line_color=colours[o],
                        showlegend=False
                    ),
                    row= int(np.floor(c / 2)) + 1,
                    col= c % 2 + 1
                )
        
        else:
            for o, out in enumerate(outputs):
                fig.add_trace(
                    go.Scatter(
                        y=out[:, c*len(age_groups)+a],
                        x=phe_parameters.regional_parameters.times,
                        mode='lines',
                        name=trace_name[o],
                        line_color=colours[o]
                    ),
                    row= int(np.floor(c / 2)) + 1,
                    col= c % 2 + 1
                )

# Add axis labels
fig.update_layout(
    boxmode='group',
    title='PHE vs SEIR Model', 
    width=900, 
    height=800,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black', title='Time (days)'),
    yaxis=dict(linecolor='black'),
    xaxis2=dict(linecolor='black', title='Time (days)'),
    yaxis2=dict(linecolor='black'),
    xaxis3=dict(linecolor='black', title='Time (days)'),
    yaxis3=dict(linecolor='black'),
    xaxis4=dict(linecolor='black', title='Time (days)'),
    yaxis4=dict(linecolor='black'),
    xaxis5=dict(linecolor='black', title='Time (days)'),
    yaxis5=dict(linecolor='black'))

fig.write_image('images/Phe_SEIR.pdf')
fig.show()